In [1]:
import pandas as pd
import glob
import re

In [2]:
ruta_carpeta = r'C:\Users\julio\Desktop\The_Bridge\Temario\EDA\Dataset\DataComex\IMPORTACIONES'

archivos_csv = glob.glob(f"{ruta_carpeta}\\*.csv")

dataframes = [pd.read_csv(archivo, sep=';', encoding='latin1') for archivo in archivos_csv]

df = pd.concat(dataframes, ignore_index=True)

In [3]:
columnas_deseadas = ['flujo_codigo', 'periodo_anio', 'pais_nombre', 'taric', 'euros']
df = df[columnas_deseadas]

In [4]:
# Filtrar columnas, caracteres no numéricos y convertir columnas
df['euros'] = df['euros'].replace(',', '.', regex=True).astype(float)
df['periodo_anio'] = df['periodo_anio'].astype(int)
df['pais_nombre'] = df['pais_nombre'].astype(str)

In [5]:
df = df[df['periodo_anio'] != 2023]

In [6]:
# Lista de nombres de países a mantener
nombres_paises = [
    'Alemania', 'Francia', 'Italia', 'China', 'Estados Unidos', 'Reino Unido',
    'Países Bajos', 'Portugal', 'Bélgica', 'Rusia', 'Argelia', 'Japón',
    'Nigeria', 'Turquía', 'Marruecos', 'Suiza', 'México', 'Arabia Saudí',
    'Polonia', 'Brasil', 'Irlanda', 'Suecia', 'India', 'Corea del Sur (Rep. de Corea)',
    'República Checa', 'Libia', 'Austria', 'Indonesia', 'Hungría', 'Dinamarca',
    'Noruega', 'Argentina', 'Vietnam', 'Finlandia', 'Bangladesh', 'Taiwán',
    'Irak', 'Canadá', 'Irán', 'Chile', 'Eslovaquia', 'Tailandia', 'Sudáfrica',
    'Rumanía', 'Perú', 'Ucrania', 'Egipto', 'Kazajstán', 'Angola', 'Malasia'
]

# Filtrar el DataFrame
df = df[df['pais_nombre'].isin(nombres_paises)]
df

,flujo_codigo,periodo_anio,pais_nombre,taric,euros
33,I,2000,Alemania,1,4.080866e+07
34,I,2000,Alemania,2,5.626962e+07
35,I,2000,Alemania,3,1.188463e+07
36,I,2000,Alemania,4,1.783630e+08
37,I,2000,Alemania,5,1.203628e+07
...,...,...,...,...,...
246129,I,1999,Vietnam,94,7.640077e+06
246130,I,1999,Vietnam,95,1.676129e+06
246131,I,1999,Vietnam,96,2.511060e+06
246132,I,1999,Vietnam,97,2.078562e+04


In [7]:
df_imp = df

In [8]:
df_imp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110233 entries, 33 to 246133
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   flujo_codigo  110233 non-null  object 
 1   periodo_anio  110233 non-null  int32  
 2   pais_nombre   110233 non-null  object 
 3   taric         110233 non-null  int64  
 4   euros         110233 non-null  float64
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 4.6+ MB


## TOP 50 PAISES DE LOS QUE MÁS SE IMPORTA

In [9]:
# Agrupar por 'pais_nombre' y 'periodo_anio', sumar 'euros'
df_top = df_imp.groupby('pais_nombre')['euros'].sum().reset_index()

# Ordenar por 'euros' en orden descendente
df_top = df_top.sort_values(by='euros', ascending=False)
df_top.reset_index(drop=True, inplace=True)

df_top

,pais_nombre,euros
0,Alemania,8.665621e+11
1,Francia,8.103511e+11
2,Italia,4.860339e+11
3,China,4.597965e+11
4,Reino Unido,3.090583e+11
5,Estados Unidos,3.012327e+11
6,Países Bajos,2.753968e+11
7,Portugal,2.289706e+11
8,Bélgica,1.645589e+11
9,Rusia,1.210957e+11


# Acumulado anual (95-22) del total los bienes exportados por país

In [10]:
# Agrupar por 'pais_nombre' y 'periodo_anio', sumar 'euros'
df_paises = df_imp.groupby(['pais_nombre', 'periodo_anio'])['euros'].sum().reset_index()

# Ordenar por 'euros' en orden descendente
df_paises = df_paises.sort_values(by='euros', ascending=False)
df_paises.reset_index(drop=True, inplace=True)

df_paises

,pais_nombre,periodo_anio,euros
0,China,2022,4.965302e+10
1,Alemania,2007,4.364503e+10
2,Alemania,2022,4.309587e+10
3,Francia,2022,4.095698e+10
4,Alemania,2018,4.023279e+10
...,...,...,...
1389,Angola,1998,2.763664e+07
1390,Kazajstán,1998,2.407269e+07
1391,Kazajstán,1997,1.252736e+07
1392,Kazajstán,1995,7.607296e+06


# Acumulado de taric por país

In [11]:
# Agrupar por 'pais_nombre' y 'taric', sumar 'euros'
df_taric = df_imp.groupby(['pais_nombre', 'taric'])['euros'].sum().reset_index()

# Ordenar por 'euros' en orden descendente
df_taric = df_taric.sort_values(by='euros', ascending=False)
df_taric.reset_index(drop=True, inplace=True)

df_taric

,pais_nombre,taric,euros
0,Alemania,87,2.293810e+11
1,Francia,87,2.204265e+11
2,Alemania,84,1.390346e+11
3,Argelia,27,1.134894e+11
4,Nigeria,27,1.038477e+11
...,...,...,...
4730,Angola,20,3.399000e+01
4731,Angola,11,3.000000e+01
4732,Bangladesh,13,2.108000e+01
4733,Argelia,24,1.500000e+01


In [14]:
# Agrupar por 'pais_nombre' y 'periodo_anio', sumar 'euros'
df_paises = df_imp.groupby( 'periodo_anio')['euros'].sum().reset_index()

# Ordenar por 'euros' en orden descendente
df_paises = df_paises.sort_values(by='periodo_anio', ascending=True)
df_paises.reset_index(drop=True, inplace=True)

df_paises

,periodo_anio,euros
0,1995,7.957652e+10
1,1996,8.601927e+10
2,1997,9.983533e+10
3,1998,1.125727e+11
4,1999,1.322192e+11
5,2000,1.606740e+11
6,2001,1.634653e+11
7,2002,1.653614e+11
8,2003,1.753076e+11
9,2004,1.975499e+11


In [13]:
# Agrupar por 'pais_nombre' y 'periodo_anio', sumar 'euros'
df_taric_anio = df_imp.groupby(['periodo_anio', 'taric'])['euros'].sum().reset_index()

# Ordenar por 'euros' en orden descendente
df_taric_anio = df_taric_anio.sort_values(by='euros', ascending=False)
df_taric_anio.reset_index(drop=True, inplace=True)

df_taric_anio

,periodo_anio,taric,euros
0,2022,27,7.873246e+10
1,2012,27,5.166921e+10
2,2013,27,4.807998e+10
3,2011,27,4.745009e+10
4,2008,27,4.535249e+10
...,...,...,...
2738,2021,98,3.625971e+06
2739,2013,98,3.528742e+06
2740,2022,98,2.225305e+06
2741,2014,98,1.661892e+06
